In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/spooky-author-identification/sample_submission.csv
/kaggle/input/spooky-author-identification/test.csv
/kaggle/input/spooky-author-identification/train.csv
/kaggle/input/spooky-author-identification/test/test.csv
/kaggle/input/spooky-author-identification/sample_submission/sample_submission.csv
/kaggle/input/spooky-author-identification/train/train.csv


In [2]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [3]:
test=pd.read_csv("/kaggle/input/spooky-author-identification/test.csv")
train=pd.read_csv("/kaggle/input/spooky-author-identification/train.csv")
sample=pd.read_csv("/kaggle/input/spooky-author-identification/sample_submission.csv")

In [4]:
#getting the length of the text and storing it as feature
train['num_feature']=train['text'].apply(len)

In [5]:
def word_count(string):
    words=string.split()
    return len(words)
train['num_words']=train['text'].apply(word_count)

In [6]:
def remove_punctuation(Text):
    translator = str.maketrans('', '', string.punctuation)
    return Text.translate(translator)
train['text'] = train['text'].apply(remove_punctuation)

In [7]:
train.head()

,id,text,author,num_feature,num_words
0,id26305,This process however afforded me no means of a...,EAP,231,41
1,id17569,It never once occurred to me that the fumbling...,HPL,71,14
2,id11008,In his left hand was a gold snuff box from whi...,EAP,200,36
3,id27763,How lovely is spring As we looked from Windsor...,MWS,206,34
4,id12958,Finding nothing else not even gold the Superin...,HPL,174,27


In [8]:
sw = stopwords.words('english')
# displaying the stopwords
np.array(sw)
def stopwords1(text):
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)
train['text'] = train['text'].apply(stopwords1)

In [9]:
stemmer = SnowballStemmer("english")
def stemming(text):    
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)
train['text'] = train['text'].apply(stemming)

In [10]:
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()
x = v.fit_transform(train['text'])

In [12]:
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
#print(df1)
res = pd.concat([train, df1], axis=1)
res=res.drop(['text'], axis=1)
res=res.drop(['id'], axis=1)
#res=res.drop(['author'], axis=1)
res=res.drop(['author'], axis=1)

In [13]:
res.head()

,num_feature,num_words,aaem,ab,aback,abaft,abandon,abaout,abas,abash,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
0,231,41,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,71,14,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,200,36,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,206,34,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,174,27,0.0,0.0,0.0,0.0,0.266318,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
res.shape

(19579, 15141)

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(res, y,
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
clf_cv=GridSearchCV(clf,param_grid,cv=5)
clf_cv.fit(X_train, y_train)
predictions = clf_cv.predict_proba(X_test)
from sklearn.metrics import log_loss
a=log_loss(y_test, predictions)
print("The log loss is:",a)

The log loss is: 0.4616788906041756


In [17]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [18]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


In [19]:
test['num_feature']=test['text'].apply(len)
test.head()

,id,text,num_feature
0,id02310,"Still, as I urged our leaving Ireland with suc...",110
1,id24541,"If a fire wanted fanning, it could readily be ...",330
2,id00134,And when they had broken down the frail door t...,189
3,id27757,While I was thinking how I should possibly man...,223
4,id04081,I am not sure to what limit his knowledge may ...,53


In [20]:
test.isnull().values.any()

False

In [21]:
test['num_words']=test['text'].apply(word_count)
test.head()
test.isnull().values.any()

False

In [22]:
test['text'] = test['text'].apply(remove_punctuation)
test['text'] = test['text'].apply(stopwords1)
test['text'] = test['text'].apply(stemming)
test.isnull().values.any()

False

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

con = TfidfVectorizer()
conf = con.fit_transform(test['text'])

In [24]:
df3 = pd.DataFrame(conf.toarray(), columns=con.get_feature_names())
print(df3)

       ab  abaddon  abandon  abaout  abat  abbey  abdic  abdomen  abdul  \
0     0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
1     0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
2     0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
3     0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
4     0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
...   ...      ...      ...     ...   ...    ...    ...      ...    ...   
8387  0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
8388  0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
8389  0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
8390  0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   
8391  0.0      0.0      0.0     0.0   0.0    0.0    0.0      0.0    0.0   

      aberr  ...  zodiac  zoilus  zone  zuro  äeriform  æmilianus  æronaut  \
0       0.0  ...     

In [25]:
rest1 = pd.concat([test, df3], axis=1)
rest1.head()
rest1.isnull().values.any()

False

In [26]:
rest1.head()

,id,text,num_feature,num_words,ab,abaddon,abandon,abaout,abat,abbey,...,zodiac,zoilus,zone,zuro,äeriform,æmilianus,æronaut,æstheticus,été,être
0,id02310,still urg leav ireland inquietud impati father...,110,19,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,id24541,fire want fan could readili fan newspap govern...,330,62,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,id00134,broken frail door found two clean pick human s...,189,33,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,id27757,think possibl manag without one actual tumbl h...,223,41,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,id04081,sure limit knowledg may extend,53,11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
rest1.isnull().values.any()

False

In [28]:
rest1=rest1.drop(['text'], axis=1)
rest1=rest1.drop(['id'], axis=1)
rest1.head()

,num_feature,num_words,ab,abaddon,abandon,abaout,abat,abbey,abdic,abdomen,...,zodiac,zoilus,zone,zuro,äeriform,æmilianus,æronaut,æstheticus,été,être
0,110,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,330,62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,189,33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,223,41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,53,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
rest1.shape

(8392, 10982)

In [30]:
test_shaped = rest1.reindex(columns = res.columns, fill_value=0)

In [31]:
test_shaped.shape

(8392, 15141)

In [32]:
submission_predict1=clf_cv.predict_proba(test_shaped)
print(submission_predict1)

[[0.03432926 0.03119716 0.93447358]
 [0.8106854  0.10459246 0.08472214]
 [0.39574169 0.58456427 0.01969404]
 ...
 [0.73455303 0.02782967 0.2376173 ]
 [0.10071215 0.01499554 0.88429232]
 [0.13926482 0.84414855 0.01658663]]


In [33]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


In [34]:
type(submission_predict1)

numpy.ndarray

In [35]:
sub_dataset = pd.DataFrame({'EAP': submission_predict1[:, 0], 'HPL': submission_predict1[:, 1],'MWS': submission_predict1[:, 2]})
sub_dataset.head()

,EAP,HPL,MWS
0,0.034329,0.031197,0.934474
1,0.810685,0.104592,0.084722
2,0.395742,0.584564,0.019694
3,0.376753,0.618428,0.004819
4,0.764435,0.122882,0.112683


In [36]:
test.head()

,id,text,num_feature,num_words
0,id02310,still urg leav ireland inquietud impati father...,110,19
1,id24541,fire want fan could readili fan newspap govern...,330,62
2,id00134,broken frail door found two clean pick human s...,189,33
3,id27757,think possibl manag without one actual tumbl h...,223,41
4,id04081,sure limit knowledg may extend,53,11


In [37]:
test=test.drop(['text'], axis=1)
test=test.drop(['num_feature'], axis=1)
test=test.drop(['num_words'], axis=1)
test.head()

,id
0,id02310
1,id24541
2,id00134
3,id27757
4,id04081


In [38]:
test.shape

(8392, 1)

In [39]:
a=pd.concat([test,sub_dataset], axis=1)
type(a)

pandas.core.frame.DataFrame

In [40]:
a.head()

,id,EAP,HPL,MWS
0,id02310,0.034329,0.031197,0.934474
1,id24541,0.810685,0.104592,0.084722
2,id00134,0.395742,0.584564,0.019694
3,id27757,0.376753,0.618428,0.004819
4,id04081,0.764435,0.122882,0.112683


In [41]:
a.shape

(8392, 4)

In [42]:
filename = 'sub.csv'

a.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: sub.csv
